Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [14]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
start = time.time()
very_start = time.time()

In [15]:
#import pandas as pd, 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)
import cudf, cupy, time, rmm
cudf.__version__

'0.16.0'

In [16]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [17]:
'''cluster = LocalCUDACluster(ip='10.2.61.36')#,protocol="ucx", 
                           #rmm_pool_size="26GB",
                           #enable_tcp_over_ucx=True, enable_nvlink=True)
#cluster = LocalCUDACluster()
client = Client(cluster)
client'''
cluster = LocalCUDACluster()
client = Client(cluster)
client.run(cudf.set_allocator, "managed")
client

Client Scheduler: tcp://127.0.0.1:45791 Dashboard: http://127.0.0.1:42341/status,Cluster Workers: 1 Cores: 1 Memory: 109.84 GB


In [18]:
%%time
path = './dask_input/step1_output'
df = dask_cudf.read_parquet(f'{path}/*.parquet')
df, = dask.persist(df)
_ = wait(df)
df.head()

CPU times: user 516 ms, sys: 128 ms, total: 644 ms
Wall time: 14.6 s


,hashtags,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,...,retweet,retweet_comment,like,id,language,tweet_type,media,tweet_id,a_user_id,b_user_id
0,<NA>,8957DD51710FC1C3DE81C337B459C1C9,E0EEE18B1940D8AE41CEA5B15DC421D3,1581192974,3214807,0,True,1272597682,510,393,...,0,0,0,28448498,3,2,12,34312466,1717222,615098
1,<NA>,<NA>,<NA>,1581508730,108597,551,True,1379713973,60,275,...,0,0,1581520847,64296468,54,2,12,38483937,1220979,709874
2,<NA>,<NA>,<NA>,1581201226,603,1340,False,1373559819,34,79,...,0,0,0,65627264,59,0,12,34241830,1284185,995240
3,<NA>,<NA>,<NA>,1581464495,69236,882,False,1394066296,23,143,...,0,0,1581473414,66579081,54,2,12,28207885,1337565,147284
4,<NA>,<NA>,<NA>,1581195600,1911234,4327,True,1372717142,176,594,...,0,0,0,45322295,54,2,8,11378919,1486122,984386


In [19]:
%%time
def count_token(ds,token):
    not_null = ds.isnull()==0
    return ((ds.str.count(token)+1)*not_null).fillna(0)
df['len_hashtags'] = df['hashtags'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_domains']  = df['domains'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_links']    = df['links'].map_partitions(lambda ds: count_token(ds,'\t'))

df, = dask.persist(df)
_ = wait(df)
df.head()

CPU times: user 904 ms, sys: 216 ms, total: 1.12 s
Wall time: 24.1 s


,hashtags,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,...,id,language,tweet_type,media,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links
0,<NA>,8957DD51710FC1C3DE81C337B459C1C9,E0EEE18B1940D8AE41CEA5B15DC421D3,1581192974,3214807,0,True,1272597682,510,393,...,28448498,3,2,12,34312466,1717222,615098,0,1,1
1,<NA>,<NA>,<NA>,1581508730,108597,551,True,1379713973,60,275,...,64296468,54,2,12,38483937,1220979,709874,0,0,0
2,<NA>,<NA>,<NA>,1581201226,603,1340,False,1373559819,34,79,...,65627264,59,0,12,34241830,1284185,995240,0,0,0
3,<NA>,<NA>,<NA>,1581464495,69236,882,False,1394066296,23,143,...,66579081,54,2,12,28207885,1337565,147284,0,0,0
4,<NA>,<NA>,<NA>,1581195600,1911234,4327,True,1372717142,176,594,...,45322295,54,2,8,11378919,1486122,984386,0,0,0


In [20]:
def most_frequent_token(df, col, idcol, outcol, sep='\t'):
    ds = df[[idcol,col]].compute()
    tokens = ds[col].str.tokenize(sep)
    
    token_counts = ds[col].str.token_count('\t').to_frame()
    token_counts[idcol] = ds[idcol]
    del ds
    token_rows = reconstruct_token_rows(token_counts,col,idcol)
    del token_counts
    
    tokens = tokens.to_frame()
    tokens.columns = ['token']
    tokens['token'],_ = tokens['token'].factorize()
    tokens['token'] = tokens['token']+1
    tokens[idcol] = token_rows
    
    global_token_counts = tokens['token'].value_counts().reset_index()
    global_token_counts.columns = ['token','count']
    
    tokens = tokens.merge(global_token_counts,on='token',how='left')
    del global_token_counts
    
    token_counts = tokens.groupby(idcol).agg({'count':'max'}).reset_index()
    tokens = tokens.merge(token_counts,on=idcol,how='left')
    del token_counts
    mask = tokens.count_x == tokens.count_y
    tokens = tokens[mask]
    tokens = tokens.drop(['count_x','count_y'],axis=1)
    tokens = tokens.drop_duplicates(subset=['token',idcol])
    tokens = tokens.rename(columns={'token':outcol})
    
    if outcol == col:
        df = df.drop(col,axis=1)
    df = df.merge(tokens,on=idcol,how='left')
    df[outcol] = df[outcol].fillna(0)
    del tokens
    return df

def reconstruct_token_rows(token_counts,col,idcol):
    ids = cupy.asnumpy(token_counts[idcol].values)
    rep = cupy.asnumpy(token_counts[col].values)
    token_rows = np.repeat(ids,rep)
    token_rows = cupy.asarray(token_rows)
    return token_rows


In [21]:
%%time
for col in ['domains','links','hashtags']:
    df = most_frequent_token(df, col=col, idcol='id', outcol=col, sep='\t')
    df, = dask.persist(df)
    _ = wait(df)
df.head()

CPU times: user 27.4 s, sys: 23.2 s, total: 50.5 s
Wall time: 2min 51s


,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,...,media,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links,domains,links,hashtags
0,1581481769,274293,352,True,1473357345,1784,4145,False,1417718205,False,...,12,52992145,1506727,1269436,2,1,1,218447,1424330,755803
1,1581461223,2945126,32,True,1379329448,17,56,False,1443757928,False,...,4,44659405,1371914,783075,1,3,3,335707,2593260,977303
2,1581258001,1191496,453,True,1264844645,263,371,False,1278926864,False,...,4,26672542,1086695,1185313,1,0,0,0,0,2685843
3,1581095057,220580,527,True,1220626166,667,1185,False,1273487714,False,...,6,46348282,1565749,1445016,1,0,0,0,0,2535399
4,1581482104,59465,506,True,1284373697,2,9,False,1508134710,False,...,12,46204094,1220300,504633,0,1,1,379333,3455526,0


In [22]:
%%time

TRAIN_SIZE = 106254462 # TODO correct?
VALID_SIZE = 9760684

train = df[df['id']<=TRAIN_SIZE] # df['id'].min() = 1
test0 = df[(df['id']>TRAIN_SIZE)&(df['id']<=TRAIN_SIZE+VALID_SIZE)]
test1 = df[df['id']>TRAIN_SIZE+VALID_SIZE]

print(len(train),len(test0),len(test1))

106723416 9811488 9815713
CPU times: user 224 ms, sys: 92 ms, total: 316 ms
Wall time: 4.03 s


In [23]:
%%time
path = './dask_input/step2_output'

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.15 µs


In [24]:
%%time
train.to_parquet(f'{path}/train.parquet', write_index=False)
test0.to_parquet(f'{path}/valid.parquet', write_index=False)
test1.to_parquet(f'{path}/test.parquet', write_index=False)

CPU times: user 2.43 s, sys: 520 ms, total: 2.95 s
Wall time: 1min 10s


In [25]:
print('This notebook took %.1f minutes'%((time.time()-very_start)/60.))

This notebook took 4.8 minutes
